In [98]:
!pwd

/home/jupyter/spotify-tfrs


In [100]:
import os
os.chdir('/home/jupyter/spotify-tfrs') # spotify-tfrs/moverecords.sh
os.getcwd()

'/home/jupyter/spotify-tfrs'

In [101]:
%%sh
cat moverecords.sh 
./moverecords.sh

#!/bin/bash

bucket_name=spotify-beam-v3
substring_list=(
  00111.tfrecords
)

for substring in "${substring_list[@]}"; do
   gsutil cp gs://$bucket_name/v6/train_last_5/*$substring* gs://$bucket_name/v6/train_last_5_v2/
done

sh: 2: ./moverecords.sh: Permission denied


CalledProcessError: Command 'b'cat moverecords.sh \n./moverecords.sh\n'' returned non-zero exit status 126.

In [31]:
PROJECT_ID = 'hybrid-vertex'  # <--- TODO: CHANGE THIS
LOCATION = 'us-central1' 

import json
import tensorflow as tf
import tensorflow_recommenders as tfrs

from google.cloud import storage

import numpy as np
import pickle as pkl
from pprint import pprint

In [70]:
def _is_chief(task_type, task_id): 
    ''' Check for primary if multiworker training
    '''
    return (task_type == 'chief') or (task_type == 'worker' and task_id == 0) or task_type is None

# task_type_ = 'chief'
task_type_ = 'worker'
task_id_ = 1
# task_id_ = 0

print(f'task_type: {task_type_}')
print(f'task_id_: {task_id_}')

task_type: worker
task_id_: 1


In [71]:
if _is_chief(task_type_, task_id_):
    print('if _is_chief')
    print(f'task_type: {task_type_}')
    print(f'task_id_: {task_id_}')
else:
    print('else: not chief')
    print(f'task_type: {task_type_}')
    print(f'task_id_: {task_id_}')

# if _is_chief(task_type, task_id):
#     logging.info(f"_is_chief task_type:{task_type}")
#     logging.info(f"_is_chief task_id:{task_id}")
#     with vertex_ai.start_run(RUN_NAME) as my_run:
#         logging.info(f"logging metaparams")
#         my_run.log_params(metaparams)

#         logging.info(f"logging hyperparams")
#         my_run.log_params(hyperparams)

else: not chief
task_type: worker
task_id_: 1


In [11]:
# i = 0
# for elem in raw_dataset: 
#     i = i + 1
    
# i

# V3

## v3 - train

In [9]:
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/train/-00000-of-05106.tfrecords" # 8474
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/train/-00001-of-05106.tfrecords" # 8461
SAMPLE_FILE = "gs://spotify-beam-v3/v3/train/-00002-of-05106.tfrecords" # 8461
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "album_name_can"
    value {
      bytes_list {
        value: "I Love You."
      }
    }
  }
  feature {
    key: "album_name_seed_track"
    value {
      bytes_list {
        value: "LOVETAP!"
      }
    }
  }
  feature {
    key: "album_uri_can"
    value {
      bytes_list {
        value: "spotify:album:4xkM0BwLM9H2IUcbYzpcBI"
      }
    }
  }
  feature {
    key: "album_uri_seed_track"
    value {
      bytes_list {
        value: "spotify:album:59xqFRG2IgFTsZtQ73yIp6"
      }
    }
  }
  feature {
    key: "artist_followers_can"
    value {
      float_list {
        value: 8447277.0
      }
    }
  }
  feature {
    key: "artist_followers_seed_track"
    value {
      float_list {
        value: 224020.0
      }
    }
  }
  feature {
    key: "artist_genres_can"
    value {
      bytes_list {
        value: "\'modern alternative rock\', \'modern rock\', \'pop\'"
      }
    }
  }
  feature {
    key: "artist_genres_seed_track"
    value {
   

In [10]:
cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=8461>

## v3 - valid

In [18]:
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/valid/-00000-of-00510.tfrecords" # 9420
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/valid/-00001-of-00510.tfrecords" # 9420
SAMPLE_FILE = "gs://spotify-beam-v3/v3/valid/-00002-of-00510.tfrecords" # 9421
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=9421>

In [20]:
# i = 0
# for elem in raw_dataset: 
#     i = i + 1
    
# i

## v3 - dif_artist_train

In [17]:
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/dif_artist/-00000-of-05106.tfrecords" # 9410
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/dif_artist/-00001-of-05106.tfrecords" # 9396
SAMPLE_FILE = "gs://spotify-beam-v3/v3/dif_artist/-00002-of-05106.tfrecords" # 9404
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=9404>

## v3 - dif_artist_valid

In [23]:
# "gs://spotify-beam-v3/v3/dif_artist_valid/-00000-of-00510.tfrecords"

# SAMPLE_FILE = "gs://spotify-beam-v3/v3/dif_artist_valid/-00000-of-00510.tfrecords" # 9419
# SAMPLE_FILE = "gs://spotify-beam-v3/v3/dif_artist_valid/-00001-of-00510.tfrecords" # 9430
SAMPLE_FILE = "gs://spotify-beam-v3/v3/dif_artist_valid/-00002-of-00510.tfrecords" # 9415
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=9415>

# V4

## v4 - dif_artist_train

In [30]:
# 'gs://spotify-beam-v3/v4/dif_artist/-00000-of-02553.tfrecords'

# SAMPLE_FILE = "gs://spotify-beam-v3/v4/dif_artist/-00000-of-02553.tfrecords" # 18,811
# SAMPLE_FILE = "gs://spotify-beam-v3/v4/dif_artist/-00001-of-02553.tfrecords" # 18,794
SAMPLE_FILE = "gs://spotify-beam-v3/v4/dif_artist/-00002-of-02553.tfrecords" # 18,807
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=18807>

## v4 - dif_artist_valid

In [26]:
# 'gs://spotify-beam-v3/v4/dif_artist_valid/-00000-of-00255.tfrecords'

# SAMPLE_FILE = "gs://spotify-beam-v3/v4/dif_artist_valid/-00000-of-00255.tfrecords" # 18,844
# SAMPLE_FILE = "gs://spotify-beam-v3/v4/dif_artist_valid/-00001-of-00255.tfrecords" # 18,858
SAMPLE_FILE = "gs://spotify-beam-v3/v4/dif_artist_valid/-00002-of-00255.tfrecords" # 18,854
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=18854>

## v4 - train

In [43]:
# gs://spotify-beam-v3/v4/train/-00000-of-02553.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v4/train/-00000-of-02553.tfrecords" # 16859
# SAMPLE_FILE = "gs://spotify-beam-v3/v4/train/-00001-of-02553.tfrecords" # 16897
SAMPLE_FILE = "gs://spotify-beam-v3/v4/train/-00002-of-02553.tfrecords" # 16945
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=16945>

## v4 - valid

In [40]:
# gs://spotify-beam-v3/v4/valid/-00000-of-00255.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v4/valid/-00000-of-00255.tfrecords" # 18844
# SAMPLE_FILE = "gs://spotify-beam-v3/v4/valid/-00001-of-00255.tfrecords" # 18855
SAMPLE_FILE = "gs://spotify-beam-v3/v4/valid/-00002-of-00255.tfrecords" # 18853
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=18853>

# V5

## v5 - dif_artist_train

In [37]:
# gs://spotify-beam-v3/v5/dif_artist/-00000-of-03404.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v5/dif_artist/-00000-of-03404.tfrecords" # 14074
# SAMPLE_FILE = "gs://spotify-beam-v3/v5/dif_artist/-00001-of-03404.tfrecords" # 14101
SAMPLE_FILE = "gs://spotify-beam-v3/v5/dif_artist/-00002-of-03404.tfrecords" # 14115
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=14115>

## v5 - dif_artist_valid

In [34]:
# gs://spotify-beam-v3/v5/dif_artist_valid/-00000-of-00340.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v5/dif_artist_valid/-00000-of-00340.tfrecords" # 14122
# SAMPLE_FILE = "gs://spotify-beam-v3/v5/dif_artist_valid/-00001-of-00340.tfrecords" # 14137
SAMPLE_FILE = "gs://spotify-beam-v3/v5/dif_artist_valid/-00002-of-00340.tfrecords" # 14106
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=14106>

## v5 - train

In [46]:
# gs://spotify-beam-v3/v5/train/-00000-of-03404.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v5/train/-00000-of-03404.tfrecords" # 12720
# SAMPLE_FILE = "gs://spotify-beam-v3/v5/train/-00001-of-03404.tfrecords" # 12688
SAMPLE_FILE = "gs://spotify-beam-v3/v5/train/-00002-of-03404.tfrecords" # 12662
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=12662>

## v5 - valid

In [49]:
# gs://spotify-beam-v3/v5/valid/-00000-of-00340.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v5/valid/-00000-of-00340.tfrecords" # 14142
# SAMPLE_FILE = "gs://spotify-beam-v3/v5/valid/-00001-of-00340.tfrecords" # 14126
SAMPLE_FILE = "gs://spotify-beam-v3/v5/valid/-00002-of-00340.tfrecords" # 14152
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=14152>

# v6

## v6 - dif_artist_train

In [77]:
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/dif_artist/-00000-of-00610.tfrecords" # 78758
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/dif_artist/-00001-of-00610.tfrecords" # 78767
SAMPLE_FILE = "gs://spotify-beam-v3/v6/dif_artist/-00002-of-00610.tfrecords" # 78756
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

# for raw_record in raw_dataset.take(1):
#     example = tf.train.Example()
#     example.ParseFromString(raw_record.numpy())
#     print(example)

# for x in train_dataset.take(1):
#     print(x)
    
cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=78756>

## v6 - dif_artist_valid

In [80]:
# gs://spotify-beam-v3/v6/dif_artist_valid/-00000-of-00006.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v6/dif_artist_valid/-00000-of-00006.tfrecords" # 80243
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/dif_artist_valid/-00001-of-00006.tfrecords" # 80246
SAMPLE_FILE = "gs://spotify-beam-v3/v6/dif_artist_valid/-00002-of-00006.tfrecords" # 80248
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)

cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=80248>

## v6 - train

In [83]:
# gs://spotify-beam-v3/v6/train/-00000-of-00844.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train/-00000-of-00844.tfrecords" # 76672
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train/-00001-of-00844.tfrecords" # 76630
SAMPLE_FILE = "gs://spotify-beam-v3/v6/train/-00002-of-00844.tfrecords" # 76642
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)


cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=76642>

## v6 - valid

In [86]:
# gs://spotify-beam-v3/v6/valid/-00000-of-00008.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid/-00000-of-00008.tfrecords" # 81_625
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid/-00001-of-00008.tfrecords" # 81_625
SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid/-00002-of-00008.tfrecords" # 81_627
raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)


cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=81627>

## v6 - train_last_5

In [92]:
# gs://spotify-beam-v3/v6/train_last_5/-00000-of-00111.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train_last_5/-00000-of-00111.tfrecords" # 547_149
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train_last_5/-00001-of-00111.tfrecords" # 547_140
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train_last_5/-00002-of-00111.tfrecords" # 547_137

# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train_last_5/-00000-of-00844.tfrecords" #  76645
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/train_last_5/-00001-of-00844.tfrecords" #  76636
SAMPLE_FILE = "gs://spotify-beam-v3/v6/train_last_5/-00002-of-00844.tfrecords" #  76655

raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)


cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=76655>

## v6 - valid_last_5

In [97]:
# gs://spotify-beam-v3/v6/valid_last_5/-00000-of-00001.tfrecords

# SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid_last_5/-00000-of-00008.tfrecords" # 81_625
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid_last_5/-00001-of-00008.tfrecords" # 81_627
# SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid_last_5/-00002-of-00008.tfrecords" # 81_623

SAMPLE_FILE = "gs://spotify-beam-v3/v6/valid_last_5/-00000-of-00001.tfrecords" # 613001

raw_dataset = tf.data.TFRecordDataset(SAMPLE_FILE)


cnt = raw_dataset.reduce(np.int64(0), lambda x, _: x + 1)
cnt

<tf.Tensor: shape=(), dtype=int64, numpy=613001>